In [ ]:
import os

from pymongo import AsyncMongoClient
from v4vapp_backend_v2.database.db import MongoDBClient

from v4vapp_backend_v2.actions.tracked_any import tracked_any_filter
from v4vapp_backend_v2.actions.tracked_models import TrackedBaseModel
from v4vapp_backend_v2.config.setup import InternalConfig

print("Current working directory:", os.getcwd())


project_root = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2"
if os.getcwd() != project_root:
    os.chdir(project_root)
print("Current working directory:", os.getcwd())

CONFIG = InternalConfig(config_filename="config.yaml").config

old_client = MongoDBClient(
    db_conn=CONFIG.dbs_config.default_connection,
    db_name=CONFIG.dbs_config.default_name,
    db_user=CONFIG.dbs_config.default_user,
)

client = AsyncMongoClient(old_client.uri, tz_aware=True)
db = client[CONFIG.dbs_config.default_name]
rates_collection = db["rates"]
hive_ops_collection = db["hive_ops"]


In [ ]:
from pprint import pprint

await TrackedBaseModel.update_quote()
cursor = hive_ops_collection.find(
    {
        "conv": {"$exists": True},
    },
    {"_id": 0},
)
async for doc in cursor:
    try:
        op = tracked_any_filter(doc)
        if not op.conv and op.conv.fetch_date is None:
            continue
        pprint(f"{abs(op.conv.fetch_date - op.timestamp)}")
    except ValueError:
        pass